In [1]:
#default_exp hybrid

In [2]:
#export

import os
import pickle
import attr

import pandas as pd
import scipy
import numpy as np

from game_recommender import steam_data, content_based, user_based, evaluation

In [3]:
%cd ..

/home/kuba/Projects/game_recommender


In [4]:
game_dataset = steam_data.get_steam_ratings_dataset()

In [5]:
user_based_recommender = user_based.UserBasedRecommender.make_from_steam_ratings('log_hours')
content_based_recommender = content_based.ContentBasedRecommender.make_from_steam_metadata()

In [6]:
steam_df = steam_data.load_steam_df() 

In [7]:
chosen_games_substring = 'counter strike'
chosen_games_df = steam_data.get_games_by_name(steam_df, chosen_games_substring) 

In [9]:
user_item_df = game_dataset.get_user_item_df()

In [10]:
i = 10
user_id = user_based_recommender.user_similarity_searcher.df.iloc[10,0]
user_ratings_raw = user_item_df.iloc[10]
user_ratings = user_ratings_raw[~user_ratings_raw.isna()]

In [11]:
content_based_recommender.recommend_similar_games(
    user_ratings,
    n_recommended=50,
    n_similar=10
)

world of tanks blitz mmo           1.916409
tanki online                       1.738157
blockade classic                   1.729970
warface                            1.694810
planetside 2                       1.357794
counter strike source              1.176317
post scriptum                      0.961721
argo                               0.954355
squad                              0.952416
black squad                        0.862751
arma 2 operation arrowhead         0.795574
supreme commander 2                0.760804
insurgency                         0.746282
rise of liberty                    0.713363
unturned                           0.693147
counter strike nexon zombies       0.693147
raceroom racing experience         0.693147
realm of the mad god               0.693147
ricochet                           0.693147
day of defeat                      0.693147
eldevin                            0.693147
counter strike global offensive    0.638017
left 4 dead                     

In [12]:
user_based_recommender.recommend_games_from_similar_users(user_ratings=user_ratings, n_recommended=50, n_similar=10)

name
counter strike global offensive      5.171370
counter strike                       3.412184
call of duty 4 modern warfare        2.518721
counter strike nexon zombies         1.188190
counter strike source                1.072565
team fortress 2                      0.986183
dota 2                               0.821021
tera                                 0.804729
planetside 2                         0.693147
eldevin                              0.693147
realm of the mad god                 0.693147
warface                              0.657366
team fortress classic                0.650176
half life opposing force             0.650176
half life                            0.650176
half life blue shift                 0.650176
deathmatch classic                   0.637287
day of defeat                        0.637287
ricochet                             0.637287
half life 2 deathmatch               0.629588
just cause 2                         0.622983
half life 2 episode two      

In [13]:
#export


@attr.s
class HybridRecommender:
    
    content_based_recommender: content_based.ContentBasedRecommender = attr.ib()
    user_based_recommender: user_based.UserBasedRecommender = attr.ib()

    def get_weighted_recommendations(
            self,
            user_ratings,
            n_recommended,
            user_recommendation_weight,
            content_recommendation_weight,
            mean_content_recommendation_weight,
            n_similar_users=10,
            n_similar_items=10):
    
        (
            user_based_recommendations,
            content_based_recommendations,
            mean_content_based_recommendations
        ) = self.get_recommendation_groups(
            user_ratings,
            n_recommended,
            n_similar_users,
            n_similar_items
        )
        return self.weigh_recommendations(
            [
                user_recommendation_weight,
                content_recommendation_weight,
                mean_content_recommendation_weight
            ],
            [
                user_based_recommendations,
                content_based_recommendations,
                mean_content_based_recommendations
            ],
            n_recommended
        )
    
    @classmethod
    def weigh_recommendations(
            cls,
            recommendation_groups,
            weights,
            n_recommended):
        all_recommendations = pd.concat([
            weight * recommendations for (weight, recommendations) in zip(weights, recommendation_groups)
        ])
        deduplicated_recommendations = all_recommendations.groupby(all_recommendations.index).agg('mean')
        best_recommendations_indices = deduplicated_recommendations.argsort()[::-1]
        return deduplicated_recommendations[best_recommendations_indices[:n_recommended]]
        
    def get_recommendation_groups(
            self,
            user_ratings,
            n_recommended,
            n_similar_users=10,
            n_similar_items=10):
        user_based_recommendations = self.user_based_recommender.recommend_games_from_similar_users(
            user_ratings=user_ratings,
            n_recommended=n_recommended,
            n_similar=n_similar_users
        )
        content_based_recommendations = self.content_based_recommender.recommend_similar_games(
            user_ratings=user_ratings,
            n_similar=n_similar_items,
            n_recommended=n_recommended
        )
        mean_content_based_recommendations = self.content_based_recommender.recommend_mean_similar_games(
            user_ratings=user_ratings,
            n_similar=n_similar_items,
            n_recommended=n_recommended
        )
        return (
            user_based_recommendations,
            content_based_recommendations,
            mean_content_based_recommendations
        )

In [14]:
recommender = HybridRecommender(content_based_recommender, user_based_recommender)

recommendations = recommender.get_weighted_recommendations(user_ratings, 20, user_recommendation_weight=1, content_recommendation_weight=1, mean_content_recommendation_weight=1)

In [17]:
evaluation.metrics.get_recall_at(user_ratings, recommendations)

0.3

In [18]:
evaluation.metrics.get_correlation(user_ratings, recommendations, method=scipy.stats.kendalltau)

0.10891622972743417